In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 设置模型路径
model_name = "/root/autodl-tmp/model/Qwen/Qwen3-4B"  # 本地路径
# 或者使用 Hugging Face 仓库：model_name = "Qwen/Qwen3-4B"

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 使用 FP16 减少显存占用
    device_map="cuda:0",       # 指定 GPU 设备
    trust_remote_code=True     # 允许加载自定义代码
)

# 准备输入
prompt = "简单介绍大模型"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# 将输入转换为张量
model_inputs = tokenizer([text], return_tensors="pt").to("cuda:0")

# 生成文本
generated_ids = model.generate(
    model_inputs.input_ids,
    attention_mask=model_inputs.attention_mask,  # 提供注意力掩码
    max_new_tokens=512,                         # 设置合理最大生成长度
    do_sample=True,                            # 启用采样以增加生成多样性
    temperature=0.7,                           # 控制生成随机性
    top_p=0.9                                  # 控制生成多样性
)

# 解码生成的 token
output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# 提取最终回答（移除输入部分）
response = output_text[len(text):].strip()

# 打印结果
print("Generated Response:", response)